In [1]:
%load_ext autoreload
%autoreload 2

# simulate 2T and 4T Perovskite/Si IBC tandems

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pvcircuit as pvc
import ipywidgets as widgets
%matplotlib widget

In [3]:
# data Si IBC 1J, 2T
path = '../data/'
ISFH = 'ISFH_32-14.csv'
dfISFH = pd.read_csv(path+ISFH, index_col=0)
#parse
ISFH_darkV=np.array(dfISFH.index)
ISFH_darkI=np.array(dfISFH['darkI'])
ISFH_lightV=np.array(dfISFH['lightV'])
ISFH_lightI=np.array(dfISFH['lightI'])

In [4]:
# model 4T Pvk/Si tandem as 3T with Rz=0
Algt = 3.97
Adrk = 9.
PvkSi3T = pvc.Tandem3T(name='PvkSi3T', Eg_list=[1.7, 1.1], Rz=0)
PvkSi3T.top.set(Jext=0.02, area=Algt) #Pvk top
#PvkSi3T.top.set(Eg=1.70, Jext=.016, n=[1.0,2.0],J0ratio=[1580,20.0], Gsh=6.92e-7) #Pvk top
PvkSi3T.top.set(Eg=1.53, Jext=.024, n=[1.0,2.1],J0ratio=[12.6,10.0], Gsh=8.7e-7)
PvkSi3T.bot.set(Jext=(.04-PvkSi3T.top.Jext), Gsh=8.58e-4, J0ratio = [100., 10.], Rser = 0.5, lightarea=Algt, totalarea=Adrk) #Si bottom

# model 2T Pvk/Si tandem
PvkSi2T = pvc.Multi2T.copy3T(PvkSi3T, copy=False)
PvkSi2T.set(name='PvkSi2T')

# model independent Pvk and Si 2T cells 
SiIBC = pvc.Multi2T.single(PvkSi2T.j[1], copy=False)
SiIBC.set(name='SiIBC')

PVK = pvc.Multi2T.single(PvkSi2T.j[0], copy=False)
PVK.set(name='PVK')

In [5]:
ui, dax, lax = PvkSi2T.controls()
ui

In [6]:
# add data to interactive plots
#dax.plot(ISFH_darkV, ISFH_darkI, c='red', lw=0, marker='.', label='pISFH', zorder=0)
#dax.plot(ISFH_darkV, -ISFH_darkI, c='red', lw=0, marker='.', label='nISFH', zorder=0)
#lax.plot(ISFH_darkV, ISFH_darkI*1000, c='red', lw=0, marker='.', label='dISFH', zorder=0)
#lax.plot(ISFH_lightV, ISFH_lightI*1000, c='red', lw=0, marker='.', label='lISFH', zorder=0)

In [7]:
SiIBC

SiIBC: <tandem.Multi2T class>
T = 25.0 C, Rs2T= 0.5 Ω cm2

bot: <tandem.Junction class>
Eg = 1.10 eV, TC = 25.0 C
Jext = 16.0 , JLC = 7.2 mA/cm2
Gsh = 0.000858 S/cm2, Rser = 0.5 Ωcm2
lightA = 3.97 cm2, totalA = 9 cm2
pn = 1, beta = 15, gamma = 0
   n    J0ratio   J0(A/cm2) 
  db       1      1.314e-16 
 1.00    100.00   1.314e-14 
 2.00    10.00    3.625e-09 

In [8]:
PVK

PVK: <tandem.Multi2T class>
T = 25.0 C, Rs2T= 0 Ω cm2

top: <tandem.Junction class>
Eg = 1.53 eV, TC = 25.0 C
Jext = 24.0 , JLC = 0.0 mA/cm2
Gsh = 8.7e-07 S/cm2, Rser = 0 Ωcm2
lightA = 3.97 cm2, totalA = 3.97 cm2
pn = -1, beta = 0, gamma = 0
   n    J0ratio   J0(A/cm2) 
  db       1      1.352e-23 
 1.00    12.60    1.704e-22 
 2.10    10.00    3.456e-12 

In [9]:
PVK.MPP(), SiIBC.MPP(), PVK.MPP()['Pmp']+SiIBC.MPP()['Pmp']

({'Voc': 1.1774323590856905,
  'Isc': 0.09527999906092074,
  'Vmp': 1.05023487267018,
  'Imp': 0.09099621030314174,
  'Pmp': 0.095567393341189,
  'FF': 0.8518674597781558},
 {'Voc': 0.6816819432101322,
  'Isc': 0.06349278888390125,
  'Vmp': 0.580534584502412,
  'Imp': 0.05535555306054046,
  'Pmp': 0.03213581299590208,
  'FF': 0.7424771584371637},
 0.1277032063370911)

In [10]:
PvkSi3T.MPP()

MPP: <tandem.IV3T class>
{'name': 'MPP', 'meastype': 'CZ', 'shape': (1,)}
sizes(1, 1)

 #     mIro     mIzo     mIto     Vzt      Vrz      Vtr      mIA      mIB       VA       VB     mPtot    mIxhex   mIyhex   Vxhex    Vyhex  
(0,) -56.09   -34.43    90.52    1.055    0.579   -1.634   -56.09    90.52    0.579   -1.055   127.98   -103.66  -39.76    1.902    0.668  

In [11]:
print(PvkSi3T.MPP().Ptot[0]/Algt*1000)

32.23784683024895
